In [7]:
import pandas as pd
import numpy as np
import ast
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [8]:
df = pd.read_csv('midi_df/midi_df_v2.csv')
df = df[(df["duration"] >= 120) & (df["duration"] <= 300)]

In [9]:
df["token_sequence"] = df["token_sequence"].apply(ast.literal_eval)
df = df[df["token_sequence"].apply(len) > 100]

context_length = 64
target_length = 64

inputs, targets = [], []

for seq in df["token_sequence"]:
    for i in range(context_length, len(seq) - target_length, target_length):
        input_seq = seq[i - context_length:i]
        target_seq = seq[i:i + target_length]
        inputs.append(input_seq)
        targets.append(target_seq)

In [10]:
class MusicDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = [torch.tensor(seq, dtype=torch.long) for seq in inputs]
        self.targets = [torch.tensor(seq, dtype=torch.long) for seq in targets]

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

dataset = MusicDataset(inputs, targets)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [11]:
class MusicTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=4, num_layers=4):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = nn.Parameter(torch.randn(1, 512, d_model))
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.decoder = nn.Linear(d_model, vocab_size)

    def forward(self, src):
        x = self.embedding(src) + self.pos_encoder[:, :src.size(1)]
        x = self.transformer(x)
        return self.decoder(x)


In [12]:
vocab_size = max(max(seq) for seq in df["token_sequence"]) + 1
model = MusicTransformer(vocab_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(5):
    model.train()
    total_loss = 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        output = model(x)
        loss = criterion(output.view(-1, vocab_size), y.view(-1))
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss {total_loss:.4f}")


C:\Users\andyy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1: Loss 3834.5388
Epoch 2: Loss 3818.6560
Epoch 3: Loss 3816.2587
Epoch 4: Loss 3810.7306
Epoch 5: Loss 3792.7541


In [14]:
def generate(model, seed_seq, max_length=128):
    model.eval()
    generated = seed_seq[:]
    for _ in range(max_length):
        input_seq = torch.tensor([generated[-64:]], dtype=torch.long).to(device)
        with torch.no_grad():
            output = model(input_seq)
        next_token = output[0, -1].argmax().item()
        generated.append(next_token)
    return generated
